## Introduction

In this guide, we will walk you through building a Retrieval Augmented Generation (RAG) application using Couchbase Capella as the database, [gpt-4o](https://platform.openai.com/docs/models/gpt-4o) model as the large language model provided by OpenAI. We will use the [text-embedding-3-large](https://platform.openai.com/docs/guides/embeddings/embedding-models) model for generating embeddings.

This notebook demonstrates how to build a RAG system using:
- The [BBC News dataset](https://huggingface.co/datasets/RealTimeData/bbc_news_alltime) containing news articles
- Couchbase Capella as the vector store with Search Vector Index for vector index creation
- LlamaIndex framework for the RAG pipeline
- OpenAI for embeddings and text generation

We leverage Couchbase's Search Vector Index to create and manage vector indexes, enabling efficient semantic search capabilities. The Search Vector Index provides the infrastructure for storing, indexing, and querying high-dimensional vector embeddings alongside traditional text search functionality.

Semantic search goes beyond simple keyword matching by understanding the context and meaning behind the words in a query, making it an essential tool for applications that require intelligent information retrieval. This tutorial will equip you with the knowledge to create a fully functional RAG system using OpenAI Services and LlamaIndex.

## Before you start

### Create and Deploy Your Operational cluster on Capella

To get started with Couchbase Capella, create an account and use it to deploy an operational cluster.

To know more, please follow the [instructions](https://docs.couchbase.com/cloud/get-started/create-account.html). 

### Couchbase Capella Configuration

When running Couchbase using [Capella](https://cloud.couchbase.com/sign-in), the following prerequisites need to be met:

* Have a multi-node Capella cluster running the Data, Query, Index, and Search services.
* Create the [database credentials](https://docs.couchbase.com/cloud/clusters/manage-database-users.html) to access the bucket (Read and Write) used in the application.
* [Allow access](https://docs.couchbase.com/cloud/clusters/allow-ip-address.html) to the Cluster from the IP on which the application is running.

### OpenAI Models Setup

In order to create the RAG application, we need an embedding model to ingest the documents for Vector Search and a large language model (LLM) for generating the responses based on the context. 

For this implementation, we'll use OpenAI's models which provide state-of-the-art performance for both embeddings and text generation:

**Embedding Model**: We'll use OpenAI's `text-embedding-3-large` model, which provides high-quality embeddings with 3,072 dimensions for semantic search capabilities.

**Large Language Model**: We'll use OpenAI's `gpt-4o` model for generating responses based on the retrieved context. This model offers excellent reasoning capabilities and can handle complex queries effectively.

**Prerequisites for OpenAI Integration**:
* Create an OpenAI account at [platform.openai.com](https://platform.openai.com)
* Generate an API key from your OpenAI dashboard
* Ensure you have sufficient credits or a valid payment method set up
* Set up your API key as an environment variable or input it securely in the notebook

For more details about OpenAI's models and pricing, please refer to the [OpenAI documentation](https://platform.openai.com/docs/models).


## Installing Necessary Libraries
To build our RAG system, we need a set of libraries. The libraries we install handle everything from connecting to databases to performing AI tasks. Each library has a specific role: Couchbase libraries manage database operations, LlamaIndex handles AI model integrations, and we will use the OpenAI SDK for generating embeddings and calling OpenAI's language models.


In [1]:
# Install required packages
%pip install --no-user --quiet datasets==3.6.0 llama-index==0.14.13 llama-index-vector-stores-couchbase==0.6.0 llama-index-embeddings-openai==0.5.1 llama-index-llms-openai==0.6.18 python-dotenv==1.2.1


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Importing Necessary Libraries
The script starts by importing a series of libraries required for various tasks, including handling JSON, logging, time tracking, Couchbase connections, embedding generation, and dataset loading.


In [2]:
import getpass
import hashlib
import json
import logging
import os
import sys
import time
from datetime import timedelta
from dotenv import load_dotenv

from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.exceptions import CouchbaseException
from couchbase.management.buckets import CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions

from datasets import load_dataset

from llama_index.core import Settings, Document, VectorStoreIndex
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import MetadataMode
from llama_index.vector_stores.couchbase import CouchbaseSearchVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

/Users/kaustavghosh/Desktop/vector-search-cookbook/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading Sensitive Information
In this section, we prompt the user to input essential configuration settings needed. These settings include sensitive information like database credentials, collection names, and API keys. Instead of hardcoding these details into the script, we request the user to provide them at runtime, ensuring flexibility and security.

The script also validates that all required inputs are provided, raising an error if any crucial information is missing. This approach ensures that your integration is both secure and correctly configured without hardcoding sensitive information, enhancing the overall security and maintainability of your code.

**OPENAI_API_KEY** is your OpenAI API key which can be obtained from your OpenAI dashboard at [platform.openai.com](https://platform.openai.com/api-keys).

**INDEX_NAME** is the name of the Search Vector Index we will use for vector search operations.

In [3]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or getpass.getpass('Enter your OpenAI API key: ')
CB_HOST = os.getenv('CB_HOST', 'couchbase://localhost') or input('Enter Couchbase host (default: couchbase://localhost): ') or 'couchbase://localhost'
CB_USERNAME = os.getenv('CB_USERNAME', 'Administrator') or input('Enter Couchbase username (default: Administrator): ') or 'Administrator'
CB_PASSWORD = os.getenv('CB_PASSWORD', 'password') or getpass.getpass('Enter Couchbase password (default: password): ') or 'password'
CB_BUCKET_NAME = os.getenv('CB_BUCKET_NAME', 'vector-search-testing') or input('Enter Couchbase bucket name: ')
SCOPE_NAME = os.getenv('SCOPE_NAME', 'shared') or input('Enter scope name: ')
COLLECTION_NAME = os.getenv('COLLECTION_NAME', 'llamaindex') or input('Enter collection name: ')
INDEX_NAME = os.getenv('INDEX_NAME', 'vector_search_llamaindex') or input('Enter index name: ')

if not all([OPENAI_API_KEY, CB_HOST, CB_USERNAME, CB_PASSWORD, CB_BUCKET_NAME, SCOPE_NAME, COLLECTION_NAME, INDEX_NAME]):
    raise ValueError("All configuration variables must be provided.")

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

## Setting Up Logging
Logging is essential for tracking the execution of our script and debugging any issues that may arise. We set up a logger that will display information about the script's progress, including timestamps and log levels.


In [4]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)],
)
logging.getLogger("httpx").setLevel(logging.WARNING)

## Connecting to Couchbase Capella
The next step is to establish a connection to our Couchbase Capella cluster. This connection will allow us to interact with the database, store and retrieve documents, and perform vector searches.


In [5]:
try:
    # Initialize the Couchbase Cluster
    auth = PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)
    options = ClusterOptions(auth)
    
    # Connect to the cluster
    cluster = Cluster(CB_HOST, options)
    
    # Wait for the cluster to be ready
    cluster.wait_until_ready(timedelta(seconds=5))
    logging.info("Successfully connected to the Couchbase cluster")
except CouchbaseException as e:
    raise RuntimeError(f"Failed to connect to Couchbase: {str(e)}")

2026-02-10 13:28:46,976 - INFO - Successfully connected to the Couchbase cluster


## Setting Up the Bucket, Scope, and Collection
Before we can store our data, we need to ensure that the appropriate bucket, scope, and collection exist in our Couchbase cluster. The code below checks if these components exist and creates them if they don't, providing a foundation for storing our vector embeddings and documents.

In [6]:
# Create bucket if it does not exist
bucket_manager = cluster.buckets()
try:
    bucket_manager.get_bucket(CB_BUCKET_NAME)
    print(f"Bucket '{CB_BUCKET_NAME}' already exists.")
except Exception as e:
    print(f"Bucket '{CB_BUCKET_NAME}' does not exist. Creating bucket...")
    bucket_settings = CreateBucketSettings(name=CB_BUCKET_NAME, ram_quota_mb=500)
    bucket_manager.create_bucket(bucket_settings)
    print(f"Bucket '{CB_BUCKET_NAME}' created successfully.")

# Create scope and collection if they do not exist
collection_manager = cluster.bucket(CB_BUCKET_NAME).collections()
scopes = collection_manager.get_all_scopes()
scope_exists = any(scope.name == SCOPE_NAME for scope in scopes)

if scope_exists:
    print(f"Scope '{SCOPE_NAME}' already exists.")
else:
    print(f"Scope '{SCOPE_NAME}' does not exist. Creating scope...")
    collection_manager.create_scope(SCOPE_NAME)
    print(f"Scope '{SCOPE_NAME}' created successfully.")

collections = [collection.name for scope in scopes if scope.name == SCOPE_NAME for collection in scope.collections]
collection_exists = COLLECTION_NAME in collections

if collection_exists:
    print(f"Collection '{COLLECTION_NAME}' already exists in scope '{SCOPE_NAME}'.")
else:
    print(f"Collection '{COLLECTION_NAME}' does not exist in scope '{SCOPE_NAME}'. Creating collection...")
    collection_manager.create_collection(collection_name=COLLECTION_NAME, scope_name=SCOPE_NAME)
    print(f"Collection '{COLLECTION_NAME}' created successfully.")

Bucket 'vector-search-testing' already exists.
Scope 'shared' already exists.
Collection 'llamaindex' already exists in scope 'shared'.


## Creating or Updating Search Indexes
With the index definition loaded, the next step is to create or update the Vector Search Index in Couchbase. This step is crucial because it optimizes our database for vector similarity search operations, allowing us to perform searches based on the semantic content of documents rather than just keywords. By creating or updating a Vector Search Index, we enable our RAG to handle complex queries that involve finding semantically similar documents using vector embeddings, which is essential for a robust RAG system.


In [7]:
# Create search index from search_index.json file at scope level
with open('fts_index.json', 'r') as search_file:
    search_index_definition = SearchIndex.from_json(json.load(search_file))
    
    # Update search index definition with user inputs
    search_index_definition.name = INDEX_NAME
    search_index_definition.source_name = CB_BUCKET_NAME
    
    # Update types mapping
    old_type_key = next(iter(search_index_definition.params['mapping']['types'].keys()))
    type_obj = search_index_definition.params['mapping']['types'].pop(old_type_key)
    search_index_definition.params['mapping']['types'][f"{SCOPE_NAME}.{COLLECTION_NAME}"] = type_obj

    search_index_name = search_index_definition.name

    # Get scope-level search manager
    scope_search_manager = cluster.bucket(CB_BUCKET_NAME).scope(SCOPE_NAME).search_indexes()
    
    # If index exists, copy its UUID so upsert can update it
    try:
        existing_index = scope_search_manager.get_index(search_index_name)
        search_index_definition.uuid = existing_index.uuid
        logging.info(f"Found existing search index '{search_index_name}', updating...")
    except Exception:
        logging.info(f"Search index '{search_index_name}' does not exist, creating...")

    try:
        scope_search_manager.upsert_index(search_index_definition)
        logging.info(f"Search index '{search_index_name}' upserted successfully at scope level.")
    except Exception as e:
        raise RuntimeError(f"Failed to upsert search index '{search_index_name}': {str(e)}")

2026-02-10 13:28:47,000 - INFO - Found existing search index 'vector_search_llamaindex', updating...
2026-02-10 13:28:47,023 - INFO - Search index 'vector_search_llamaindex' upserted successfully at scope level.


## Load the BBC News Dataset
To build a RAG engine, we need data to search through. We use the [BBC Realtime News dataset](https://huggingface.co/datasets/RealTimeData/bbc_news_alltime), a dataset with up-to-date BBC news articles grouped by month. This dataset contains articles that were created after the LLM was trained. It will showcase the use of RAG to augment the LLM. 

The BBC News dataset's varied content allows us to simulate real-world scenarios where users ask complex questions, enabling us to fine-tune our RAG's ability to understand and respond to various types of queries.


In [8]:
try:
    news_dataset = load_dataset('RealTimeData/bbc_news_alltime', '2024-12', split="train")
    print(f"Loaded the BBC News dataset with {len(news_dataset)} rows")
except Exception as e:
    raise ValueError(f"Error loading TREC dataset: {str(e)}")

2026-02-10 13:28:47,937 - WARNING - Warning: You are sending unauthenticated requests to the HF Hub. Please set a HF_TOKEN to enable higher rate limits and faster downloads.
Loaded the BBC News dataset with 2687 rows


## Preview the Data

In [9]:
# Print the first two examples from the dataset
print("Dataset columns:", news_dataset.column_names)
print("\nFirst two examples:")
print(news_dataset[:2])

Dataset columns: ['title', 'published_date', 'authors', 'description', 'section', 'content', 'link', 'top_image']

First two examples:
{'title': ["Pakistan protest: Bushra Bibi's march for Imran Khan disappeared - BBC News", 'Lockdown DIY linked to Walleys Quarry gases - BBC News'], 'published_date': ['2024-12-01', '2024-12-01'], 'authors': ['https://www.facebook.com/bbcnews', 'https://www.facebook.com/bbcnews'], 'description': ["Imran Khan's third wife guided protesters to the heart of the capital - and then disappeared.", 'An academic says an increase in plasterboard sent to landfill could be behind a spike in smells.'], 'section': ['Asia', 'Stoke & Staffordshire'], 'content': ['Bushra Bibi led a protest to free Imran Khan - what happened next is a mystery\n\nImran Khan\'s wife, Bushra Bibi, encouraged protesters into the heart of Pakistan\'s capital, Islamabad\n\nA charred lorry, empty tear gas shells and posters of former Pakistan Prime Minister Imran Khan - it was all that remaine

## Preparing the Data for RAG

We need to extract the context passages from the dataset to use as our knowledge base for the RAG system.

In [10]:
try:
    news_articles = news_dataset
    unique_articles = {}

    for article in news_articles:
        content = article.get("content")
        if content:
            content_hash = hashlib.md5(content.encode()).hexdigest()  # Generate hash of content
            if content_hash not in unique_articles:
                unique_articles[content_hash] = article  # Store full article

    unique_news_articles = list(unique_articles.values())  # Convert back to list
    logging.info(f"We have {len(unique_news_articles)} unique articles in our database.")
except Exception as e:
    raise ValueError(f"Error preparing data: {str(e)}")

2026-02-10 13:28:51,279 - INFO - We have 1749 unique articles in our database.


## Creating Embeddings using OpenAI
Embeddings are numerical representations of text that capture semantic meaning. Unlike keyword-based search, embeddings enable semantic search to understand context and retrieve documents that are conceptually similar even without exact keyword matches. We'll use OpenAI's `text-embedding-3-large` model to create high-quality embeddings with 3,072 dimensions. This model transforms our text data into vector representations that can be efficiently searched, with a batch size of 30 for optimal processing.


In [11]:
try:
    # Set up the embedding model
    embed_model = OpenAIEmbedding(
        api_key=OPENAI_API_KEY,
        embed_batch_size=30,
        model="text-embedding-3-large"
    )
    
    # Configure LlamaIndex to use this embedding model
    Settings.embed_model = embed_model
    print("Successfully created embedding model")
except Exception as e:
    raise ValueError(f"Error creating embedding model: {str(e)}")

Successfully created embedding model


## Testing the Embeddings Model
We can test the embeddings model by generating an embedding for a string

In [12]:
try:
    test_embedding = embed_model.get_text_embedding("this is a test sentence")
    logging.info(f"Embedding dimension: {len(test_embedding)}")
except Exception as e:
    raise ValueError(f"Error testing embedding model: {str(e)}")

2026-02-10 13:28:51,751 - INFO - Embedding dimension: 3072


## Setting Up the Couchbase Vector Store
The vector store is set up to store the documents from the dataset. The vector store is essentially a database optimized for storing and retrieving high-dimensional vectors.

In [13]:
try:
    # Create the Couchbase vector store
    vector_store = CouchbaseSearchVectorStore(
        cluster=cluster,
        bucket_name=CB_BUCKET_NAME,
        scope_name=SCOPE_NAME,
        collection_name=COLLECTION_NAME,
        index_name=INDEX_NAME,
    )
    print("Successfully created vector store")
except Exception as e:
    raise ValueError(f"Failed to create vector store: {str(e)}")

Successfully created vector store


## Creating LlamaIndex Documents
In this section, we'll process our news articles and create LlamaIndex Document objects.
Each Document is created with specific metadata and formatting templates to control what the LLM and embedding model see.
We'll observe examples of the formatted content to understand how the documents are structured.

In [14]:
llama_documents = []
# Process and store documents
for article in unique_news_articles:  # Limit to first 100 for demo
    try:
        document = Document(
            text=article["content"],
            metadata={
                "title": article["title"],
                "description": article["description"],
                "published_date": article["published_date"],
                "link": article["link"],
            },
            excluded_llm_metadata_keys=["description"],
            excluded_embed_metadata_keys=["description", "published_date", "link"],
            metadata_template="{key}=>{value}",
            text_template="Metadata: \n{metadata_str}\n-----\nContent: {content}",
        )
        llama_documents.append(document)
    except Exception as e:
        print(f"Failed to save document to vector store: {str(e)}")
        continue

# Observing an example of what the LLM and Embedding model receive as input
print("The LLM sees this:")
print(llama_documents[0].get_content(metadata_mode=MetadataMode.LLM))
print("The Embedding model sees this:")
print(llama_documents[0].get_content(metadata_mode=MetadataMode.EMBED))

The LLM sees this:
Metadata: 
title=>Pakistan protest: Bushra Bibi's march for Imran Khan disappeared - BBC News
published_date=>2024-12-01
link=>http://www.bbc.co.uk/news/articles/cvg02lvj1e7o
-----
Content: Bushra Bibi led a protest to free Imran Khan - what happened next is a mystery

Imran Khan's wife, Bushra Bibi, encouraged protesters into the heart of Pakistan's capital, Islamabad

A charred lorry, empty tear gas shells and posters of former Pakistan Prime Minister Imran Khan - it was all that remained of a massive protest led by Khan’s wife, Bushra Bibi, that had sent the entire capital into lockdown. Just a day earlier, faith healer Bibi - wrapped in a white shawl, her face covered by a white veil - stood atop a shipping container on the edge of the city as thousands of her husband’s devoted followers waved flags and chanted slogans beneath her. It was the latest protest to flare since Khan, the 72-year-old cricketing icon-turned-politician, was jailed more than a year ago aft

## Creating and Running the Ingestion Pipeline

In this section, we'll create an ingestion pipeline to process our documents. The pipeline will:

1. Split the documents into smaller chunks (nodes) using the SentenceSplitter
2. Generate embeddings for each node using our embedding model
3. Store these nodes with their embeddings in our Couchbase vector store

This process transforms our raw documents into a searchable knowledge base that can be queried semantically.

In [15]:

# Process documents: split into nodes, generate embeddings, and store in vector database
# Step 3: Create and Run IndexPipeline
index_pipeline = IngestionPipeline(
    transformations=[SentenceSplitter(),embed_model], 
    vector_store=vector_store,
)

try:
    nodes = index_pipeline.run(documents=llama_documents)
    logging.info(f"Successfully ingested {len(nodes)} nodes into the vector store")
except Exception as e:
    logging.error(f"Failed to run ingestion pipeline: {str(e)}")

2026-02-10 13:30:00,717 - INFO - Successfully ingested 2329 nodes into the vector store


## Using OpenAI's Large Language Model (LLM)
Large language models are AI systems that are trained to understand and generate human language. We'll be using OpenAI's `gpt-4o` model to process user queries and generate meaningful responses based on the retrieved context from our Couchbase vector store. This model is a key component of our RAG system, allowing it to go beyond simple keyword matching and truly understand the intent behind a query. By integrating OpenAI's LLM, we equip our RAG system with the ability to interpret complex queries, understand the nuances of language, and provide more accurate and contextually relevant responses.

The language model's ability to understand context and generate coherent responses is what makes our RAG system truly intelligent. It can not only find the right information but also present it in a way that is useful and understandable to the user.

The LLM is configured using LlamaIndex's OpenAI-like provider with OpenAI's API endpoint and your OpenAI API key for seamless integration with their services.

In [16]:
try:
    # Set up the LLM
    llm = OpenAI(
        api_key=OPENAI_API_KEY,
        model="gpt-4o",
        
    )
    
    # Configure LlamaIndex to use this LLM
    Settings.llm = llm
    logging.info("Successfully created the OpenAI LLM")
except Exception as e:
    raise ValueError(f"Error creating OpenAI LLM: {str(e)}")

2026-02-10 13:30:00,722 - INFO - Successfully created the OpenAI LLM


## Creating the Vector Store Index

In this section, we'll create a VectorStoreIndex from our Couchbase vector store. This index serves as the foundation for our RAG system, enabling semantic search capabilities and efficient retrieval of relevant information.

The VectorStoreIndex provides a high-level interface to interact with our vector store, allowing us to:
1. Perform semantic searches based on user queries
2. Retrieve the most relevant documents or chunks
3. Generate contextually appropriate responses using our LLM


In [17]:
try:
    # Create your index
    index = VectorStoreIndex.from_vector_store(vector_store)
    rag = index.as_query_engine()
    logging.info("Successfully created the vector store index and query engine")
except Exception as e:
    raise ValueError(f"Error creating vector store index: {str(e)}")

2026-02-10 13:30:00,727 - INFO - Successfully created the vector store index and query engine


## Retrieval-Augmented Generation (RAG) with Couchbase and LlamaIndex

Let's test our RAG system by performing a semantic search on a sample query. In this example, we'll use a question about Pep Guardiola's reaction to Manchester City's recent form. The RAG system will:

1. Process the natural language query
2. Search through our vector database for relevant information
3. Retrieve the most semantically similar documents
4. Generate a comprehensive response using the LLM

This demonstrates how our system combines the power of vector search with language model capabilities to provide accurate, contextual answers based on the information in our database.

In [18]:
# Sample query from the dataset

query = "What was Pep Guardiola's reaction to Manchester City's recent form?"

try:
    # Perform the semantic search
    start_time = time.time()
    response = rag.query(query)
    search_elapsed_time = time.time() - start_time

    # Display search results
    print(f"\nSemantic Search Results (completed in {search_elapsed_time:.2f} seconds):")
    print(response)

except Exception as e:
    raise RuntimeError(f"Error performing semantic search: {e}")


Semantic Search Results (completed in 4.12 seconds):
Pep Guardiola has been experiencing self-doubt and has been deeply affected by Manchester City's recent poor form. He has not been sleeping well and has been visibly agitated, as seen in his interactions with the media. Guardiola has been reflecting on the situation, discussing it with those close to him, and trying to understand the reasons behind the team's struggles. He acknowledges the challenges and the possibility of not qualifying for the Champions League, expressing concern over the team's current standing and the errors being made by players. Despite the difficulties, he is seeking support from those around him to overcome his insecurities and address the issues facing the team.


## Conclusion
In this tutorial, we've built a Retrieval Augmented Generation (RAG) system using Couchbase Capella, Openai and LlamaIndex. We used the BBC News dataset, which contains real-time news articles, to demonstrate how RAG can be used to answer questions about current events and provide up-to-date information that extends beyond the LLM's training data.

The key components of our RAG system include:

1. **Couchbase Capella** as the vector database for storing and retrieving document embeddings
2. **LlamaIndex** as the framework for connecting our data to the LLM
3. **OpenAI Services** for generating embeddings (`text-embedding-3-large`) and LLM responses (`gpt-4o`)

This approach allows us to enhance the capabilities of large language models by grounding their responses in specific, up-to-date information from our knowledge base. 